In [2]:
!pip install flask-ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from flask import Flask, jsonify, request, url_for, render_template, flash, request, session, g
from werkzeug.utils import redirect
from flask_ngrok import run_with_ngrok
from bs4 import BeautifulSoup
import urllib.request
import base64
from io import BytesIO


app = Flask(__name__)
run_with_ngrok(app)

@app.route("/test", methods=['POST'])
def test():
    if request.method == 'POST':

        return "<h1> Hello Electron <h1>"

def result(description):
    
    if request.form.get('mint-pic'):
        return redirect(url_for('gen.done', description=description))

    device = 'cpu' # gpu 달려있으면  cuda 셋팅 후 전환
    set_seed(0)

    prompt = description # 텍스트 데이터를 여기서 수신 받아야함  promptRx

    model = Dalle.from_pretrained('minDALL-E/1.3B')  # This will automatically download the pretrained model.
    model.to(device=device)

    # Sampling
    images = model.sampling(prompt=prompt,
                            top_k=256, # It is recommended that top_k is set lower than 256.
                            top_p=4,
                            softmax_temperature=1,
                            num_candidates=4, # device 성능에 의존. 수치를 낮출수록 생성속도가 빨ㄹ라짐
                            device=device).cpu().numpy()
    images = np.transpose(images, (0, 2, 3, 1))

    # CLIP Re-ranking
    model_clip, preprocess_clip = clip.load("ViT-B/32", device=device)
    model_clip.to(device=device)
    rank = clip_score(prompt=prompt,
                    images=images,
                    model_clip=model_clip,
                    preprocess_clip=preprocess_clip,
                    device=device)

    # Plot images ==> 결과 데이터를 여기서 보내줘야 함
    images = images[rank]
    plt.imshow(images[0]) # imgTx
    plt.axis('off')
    # plt.show()
    buf = BytesIO()
    # fname = str(hash(id(Figure[0])))
    plt.savefig(buf, format="png")
    imgname = description.replace(' ','_')+'.png'
    plt.savefig('app/static/gen_result/'+imgname)
    #Embed the result in the html output
    data = base64.b64encode(buf.getbuffer()).decode("ascii")
    data = f'data:imagegen/png;base64,{data}'
        # "./static/imagegen/"+fname+".png"
    # plt.imshow(images[0]) # imgTx
    # Image.fromarray((images[i]*255).astype(np.uint8))
    return render_template('gen/result.html', fname=imgname, title=description) # --> api call --> react receive 

app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://b552-104-155-204-13.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [13/Sep/2022 11:46:16] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [13/Sep/2022 11:46:16] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [13/Sep/2022 11:46:17] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [13/Sep/2022 11:46:18] "GET /favicon.ico HTTP/1.1" 404 -
